# Fully Connected

In [1]:
import torch as th
import warnings

warnings.filterwarnings("ignore")
th.backends.quantized.engine = "qnnpack"  # for ARM CPU
th.manual_seed(0)

In [2]:
class Model(th.nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size=3,
        stride=1,
        padding=1,
        dilation=1,
        groups=1,
        bias=False,
    ):
        super().__init__()
        self.quant = th.ao.quantization.QuantStub()
        self.conv = th.nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            dilation,
            groups,
            bias,
        )
        self.dequant = th.ao.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = self.conv(x)
        x = self.dequant(x)
        return x

In [3]:
input_channels = 2
output_channels = 2
input_width = 3
input_height = 3
kernel_size = 3
stride = 1
padding = 1
dilation = 1
groups = 1
bias = False
m = Model(
    input_channels,
    output_channels,
    kernel_size,
    stride,
    padding,
    dilation,
    groups,
    bias=bias,
)

m.qconfig = th.ao.quantization.QConfig(
    activation=th.ao.quantization.MovingAverageMinMaxObserver.with_args(
        quant_min=-128,
        quant_max=127,
        dtype=th.qint8,
        qscheme=th.per_tensor_symmetric,
        reduce_range=False,
    ),
    weight=th.ao.quantization.MovingAverageMinMaxObserver.with_args(
        quant_min=-128,
        quant_max=127,
        dtype=th.qint8,
        qscheme=th.per_tensor_symmetric,
        reduce_range=False,
    ),
)

# Prepare
pm = th.ao.quantization.prepare_qat(m)

# Train
pm(th.rand(32, input_channels, input_width, input_height))

# Convert
qm = th.ao.quantization.convert(pm.eval())

In [4]:
# Test
x = th.rand(1, input_channels, input_width, input_height)
xq = qm.quant(x)
y = qm(x)
yq = th.round(y / qm.conv.scale)
pdims = (0, 2, 3, 1)  # permutation dimensions

print(f"Float input: {x.flatten()}\n")
print(f"Quantized input: {xq.int_repr()}\n")
print(f"Permuted Quantized input: {xq.int_repr().permute(pdims).flatten()}\n")
print(f"Float output: {y}\n")
print(f"Quantized output: {yq}\n")
print(f"Permuted Quantized output: {yq.permute(pdims).flatten()}\n")
print(
    f"Multiplier: {th.round((qm.quant.scale * qm.conv.weight().q_scale() / qm.conv.scale) * (2**31))}\n"
)
print(f"Quantized weights: {qm.conv.weight().int_repr().int()}\n")
print(
    f"Permuted Quantized weights: {qm.conv.weight().int_repr().permute(pdims).flatten().int()}\n"
    # f"Permuted Quantized weights: {qm.conv.weight().int_repr().flatten()}\n"
)
print(
    f"Quantized bias: {th.round(qm.conv.bias() / (qm.quant.scale * qm.conv.weight().q_scale())).int()}\n"
)

Float input: tensor([0.1022, 0.3797, 0.7720, 0.2957, 0.9200, 0.1559, 0.0801, 0.2745, 0.5808,
        0.9604, 0.2613, 0.6788, 0.3746, 0.3916, 0.8677, 0.1125, 0.5531, 0.9702])

Quantized input: tensor([[[[ 13,  48,  98],
          [ 38, 117,  20],
          [ 10,  35,  74]],

         [[122,  33,  87],
          [ 48,  50, 111],
          [ 14,  71, 124]]]], dtype=torch.int8)

Permuted Quantized input: tensor([ 13, 122,  48,  33,  98,  87,  38,  48, 117,  50,  20, 111,  10,  14,
         35,  71,  74, 124], dtype=torch.int8)

Float output: tensor([[[[-0.0336,  0.0056, -0.1902],
          [-0.1063, -0.2182, -0.1287],
          [-0.2462, -0.1678, -0.3972]],

         [[ 0.2686, -0.2574, -0.0671],
          [-0.0504, -0.5147, -0.2014],
          [-0.0504, -0.2630, -0.2070]]]])

Quantized output: tensor([[[[ -6.,   1., -34.],
          [-19., -39., -23.],
          [-44., -30., -71.]],

         [[ 48., -46., -12.],
          [ -9., -92., -36.],
          [ -9., -47., -37.]]]])

Permuted Qua

In [5]:
one_two = qm.conv.weight()

In [6]:
one_two.int_repr().flatten()

tensor([  -1,   72, -110,  -98,  -51,   36,   -3,  106,  -12,   35,  -40,  -26,
        -128,  -88,  -55,    5,   53,   80,  -90,  -58,   48,  111,  -27,  100,
         -22,   14,  121, -124,  -84,  -34,  -52,  115,  -87,  -61,  -93, -125],
       dtype=torch.int8)

In [7]:
one_two.int_repr(), two_one.int_repr(), one_two.shape, two_one.shape

NameError: name 'two_one' is not defined

In [ ]:
dims = (0, 2, 3, 1)
print(one_two.int_repr().permute(dims).flatten())
print(two_one.int_repr().permute(dims).flatten())

In [ ]:
two_one = qm.conv.weight()

In [ ]:
two_one.permute(2, 3, 0, 1).flatten().int_repr()

In [ ]:
xq.int_repr().flatten()

In [ ]:
qm.conv.weight().int_repr().flatten()